In [1]:
!pip install spotipy

In [0]:
# Import libraries

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [0]:
# Setup credentials 

client_id = '055ab63103814554b2aa174b1e3de7af'
client_secret = 'd5309d21a07746de8a363dc7c053b75d'
redirect_uri = 'http://localhost/'

In [0]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace = False

In [0]:
# Function for finding similar songs

def dj_rec(track_id, max_distance=6.5, neighbors=3):
    """
    Prints the ids of relevant songs, along with their distance from the input song.

    Parameters:
    track_id (string): Spotify track id.
    max_distance (float): maximum euclidean distance a song can be 
                          from the input song for it to be returned.
    neighbors (int): number of song recommendations returned.
    """

    rel_artists = sp.artist_related_artists(sp.track(track_id=track_id)['artists'][0]['id'])['artists']
    artist_log = []
    for a in rel_artists:
        artist_log.append(a['id'])
    feat_log = []
    for artist in artist_log:
        for track in sp.artist_top_tracks(artist)['tracks']:
            feat_log.append(sp.audio_features(track['id'])[0])
    
    catalog = pd.DataFrame.from_dict(feat_log)
    
    root = pd.DataFrame.from_dict(sp.audio_features(tracks=[track_id]))

    merged_df = root.append(catalog, ignore_index=True)
    
    dropped_df = merged_df.drop(columns=['uri', 'track_href', 'id', 'duration_ms', 'time_signature', 'mode', 'loudness', 'type', 'analysis_url'])
    scaled_df = StandardScaler().fit_transform(dropped_df)
    trans_array = scaled_df.copy()
    trans_array[:,0] = [u*2.4 for u in trans_array[:,0]] # acousticness
    trans_array[:,1] = [((u*u)**0.5)*u for u in trans_array[:,1]] # danceability
    trans_array[:,2] = [u*1.7 for u in trans_array[:,2]] # energy
    trans_array[:,3] = [u*1.4 for u in trans_array[:,3]] # instrumentalness
    trans_array[:,4] = [u*0.9 for u in trans_array[:,4]] # key
    trans_array[:,5] = [u*1.0 for u in trans_array[:,5]] # liveness
    trans_array[:,6] = [u*1.0 for u in trans_array[:,6]] # speechiness
    trans_array[:,7] = [u*1.1 for u in trans_array[:,7]] # tempo
    trans_array[:,8] = [u*2.5 for u in trans_array[:,8]] # valence

    knn = NearestNeighbors()
    knn.fit(trans_array)

    rec = knn.kneighbors(trans_array[[0]], n_neighbors=neighbors+1)

    print('Seed')
    print('ID:     ', root.loc[0,'id'], '\n')
    print('\nRecommendations')

    for n in range(1,neighbors+1):
        if rec[0][0][n] <= max_distance:
            print('ID:      ', merged_df.loc[rec[1][0][n],'id'])
            print('Distance:', rec[0][0][n], '\n')

    if rec[0][0][1] > max_distance:
        print('No matches in catalog')

In [8]:
# Example

dj_rec('35B5N7Re78hqgqc2cmt6Zs', max_distance=10.0, neighbors=3)

retrying ...6secs
retrying ...8secs
Seed
ID:      35B5N7Re78hqgqc2cmt6Zs 


Recommendations
ID:       0yraRKfP8x2JvzauMrMg5x
Distance: 2.1396135876233973 

ID:       2a3OvofOcwGIqUuTddaOhm
Distance: 2.388325393922539 

ID:       2RRqwGxtHGbBVtOpvpUCge
Distance: 2.615545411402499 



In [12]:
# Find song name from track id

sp.track('35B5N7Re78hqgqc2cmt6Zs')['name']

'Moon Two'

In [11]:
# Find artist name from track id

sp.track('35B5N7Re78hqgqc2cmt6Zs')['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Fw9V88d2NoplsZBOkBWaS'},
  'href': 'https://api.spotify.com/v1/artists/4Fw9V88d2NoplsZBOkBWaS',
  'id': '4Fw9V88d2NoplsZBOkBWaS',
  'name': 'Chris Carter',
  'type': 'artist',
  'uri': 'spotify:artist:4Fw9V88d2NoplsZBOkBWaS'}]